In [1]:
 import numpy as np

In [2]:
#случайные точки из двумерного нормального распределения
mu_0 = -1
mu_1 = 1
d = 1
x_0 = np.random.normal(mu_0, d, (1000,2))
x_1 = np.random.normal(mu_1, d, (1000,2))
x_0, x_1

(array([[-0.26771515, -1.75427522],
        [-2.72828431,  1.51779955],
        [ 0.79919503, -2.12949038],
        ...,
        [-0.15767111, -0.41202009],
        [-1.32606767, -0.13785321],
        [-0.82686214, -1.73373537]]), array([[ 0.62334382,  1.13507929],
        [ 1.51881388,  2.13425266],
        [-0.00458961,  0.85130233],
        ...,
        [ 0.95753486,  1.70509786],
        [ 1.25854232,  1.41964845],
        [ 0.66499682,  0.59001694]]))

In [3]:
#создаем таргеты для точек
y_0 = np.ones(len(x_0))
y_1 = np.zeros(len(x_1))

In [5]:
#совмещаем точки в один array и перемешиваем
data_0 = np.array(list(zip(x_0,y_0)))
data_1 = np.array(list(zip(x_1,y_1)))
data = np.vstack([data_0, data_1])
np.random.shuffle(data)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  This is separate from the ipykernel package so we can avoid doing imports until


In [6]:
def sigmoid(z): 
  return 1./(1+np.exp(-z))

In [7]:
def loss(t, y): # функция потерь MSE - минимизируем мсе, чтобы полученные значения весов стремились к наиболее вероятным (Мат ожидание)
 return np.sum((t-y)**2)*0.5

In [8]:
def loss(t, y): # функция потерь log_loss
 return -(t*np.log(y)+(1-t)*np.log(1-y))

In [9]:
def diff_loss(t, y): # градиент функции потерь log_loss с ф.а. сигмоидой такой же как MSE 
  return np.sum(y-t)

In [10]:
def weights_Xavier(size): #функция Ксавьера для инициализации весов
    import math
    np.random.seed(0)
    scale = 1/max(1., (2+2)/2.)
    limit = math.sqrt(3.0 * scale)
    weights = np.random.uniform(-limit, limit, size=size)
    return weights

In [12]:
class Layer:
  def __init__(self, neurons_inp, neurons_first, neurons_out, lr=0.01):
    self.shape_first = (neurons_inp, neurons_first) 
    self.shape_out = (neurons_first, neurons_out)
    self.lr = lr
    self.w_1 = weights_Xavier(self.shape_first)
    self.w_2 = weights_Xavier(self.shape_out)
    self.b_1 = np.zeros((1, neurons_first), dtype=np.float)
    self.b_2 = np.zeros((1, neurons_out), dtype=np.float)
    self._clear_grads()

  def _clear_grads(self): # функция очистки градиентов
      self.inp = None
      self.activations = None
      self.d_sigma = None
      self.d_w = None
      self.d_b = None

  def __call__(self, x): # функция прямого прохода
      if len(x.shape) == 1:
          x = x.reshape(1, -1)
      self.inp = x
      self.activations_first = sigmoid(x.dot(self.w_1)+self.b_1) #функция активации сигмоида, перемножаем входные данные с весами и прибавляем смещение, всё это запихиваем в сигмоиду
      self.activations_second = sigmoid(self.activations_first.dot(self.w_2)+self.b_2) #второй слой
      return self.activations_second
    
  def backward(self, grad): #обратный проход, откуда град передаем из diff_loss
      self.d_sigma_1 = self.activations_first*(1 - self.activations_first) #производная сигмоиды - функции активации для первого слоя
      self.d_sigma_2 = self.activations_second*(1 - self.activations_second) #производная сигмоиды - функции активации для второго слоя
      self.d_w_1 = self.grad_w_1(grad) #производная весов первого слоя
      self.d_w_2 = self.grad_w_2(grad) #производная весов второго слоя
      self.d_b_1 = self.grad_b_1(grad) #производная смещения первого слоя
      self.d_b_2 = self.grad_b_2(grad) #производная смещения второго слоя
      result = self.grad_x(grad)
      return result #возвращаем градиент по икс от града

  def grad_w_1(self, grad): #производная весов первого слоя
      return self.w_2 * grad * self.inp * self.d_sigma_1.T * self.d_sigma_2

  def grad_w_2(self, grad):#производная весов второго слоя
      return grad * self.inp.T * self.d_sigma_2

  def grad_b_1(self, grad): #производная смещения первого слоя
      return grad * self.d_sigma_1 * self.w_2.T * self.d_sigma_2

  def grad_b_2(self, grad): #производная смещения второго слоя
      return self.d_sigma_2 * grad
  
  def grad_x(self, grad):
      return self.w_1.dot(self.w_2) * self.d_sigma_1.T * self.d_sigma_2
  
  def step(self): #обновляе веса и смещения для первого и второго слоёв
      self.w_1 = self.w_1 - self.d_w_1 * self.lr
      self.b_1 = self.b_1 - self.d_b_1 * self.lr
      self.w_2 = self.w_2 - self.d_w_2 * self.lr
      self.b_2 = self.b_2 - self.d_b_2 * self.lr
      self._clear_grads()

In [13]:
# Инициализируем сетку с 2 ейронами на входном слое, 2 на скрытом и 1 на выходе
l1 = Layer(2, 2, 1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if __name__ == '__main__':


In [14]:
# обучаем сеть 10 эпох
for i in range(10):
  for X, Y in data:
    y = l1(X)
    d_y = diff_loss(Y, y)
    l1.backward(d_y)
    l1.step()
    print('======')
    print(i)
    print('loss', d_y)
    print('pred', Y, l1(X))

Выходные данные были обрезаны до нескольких последних строк (5000).
9
loss 0.06853291803418243
pred 0.0 [[0.06852354]]
9
loss 0.05648568040284439
pred 0.0 [[0.0564791]]
9
loss 0.02736852266081798
pred 0.0 [[0.02736764]]
9
loss 0.04976007876044821
pred 0.0 [[0.04975599]]
9
loss -0.11134392509756941
pred 1.0 [[0.888669]]
9
loss 0.010350619136269864
pred 0.0 [[0.01035055]]
9
loss 0.018094904473426338
pred 0.0 [[0.01809461]]
9
loss 0.023041435619103663
pred 0.0 [[0.02304087]]
9
loss 0.2751434793942781
pred 0.0 [[0.27481013]]
9
loss -0.5612734305092721
pred 1.0 [[0.43968308]]
9
loss -0.6298680110161545
pred 1.0 [[0.37114362]]
9
loss -0.11906279891463478
pred 1.0 [[0.88095331]]
9
loss -0.11730386424827555
pred 1.0 [[0.88271181]]
9
loss -0.13713179161222666
pred 1.0 [[0.86289397]]
9
loss -0.2462581624485488
pred 1.0 [[0.75390674]]
9
loss -0.1263449578504079
pred 1.0 [[0.87367427]]
9
loss 0.012680943992304013
pred 0.0 [[0.01268082]]
9
loss 0.018802071089834414
pred 0.0 [[0.01880174]]
9
loss -0